# 00 - Provide Paths to Relevant Files

OpenAI API Key Path

In [1]:
api_key_path = "/Users/cu135/Library/CloudStorage/OneDrive-Personal/OneDrive_Documents/Work/Software/Research/nimlab/gpt_document_reader/openai_key.txt"

json_file_path is the path to the segmented JSON generated at the end of notebook 03. 

In [2]:
json_file_path = "/Users/cu135/Dropbox (Partners HealthCare)/studies/Aphantasia_Kutsche/json/_case_labeled_sections.json"

# 01 - Define Inclusion/Exclusion Questions

Examples generated below:

In [3]:
from calvin_utils.gpt_sys_review.gpt_utils import QuestionTemplate

question_template = QuestionTemplate()
question_template.inclusion_exclusion_questions()

Here are example inclusion questions:
{
    "Amnesia case report? (Y/N)": "case_report",
    "Published in English? (Y/N)": "is_english"
}
Here are example exclusion questions:
{
    "Transient amnesia, reversible amnesia symptom, severe confabulation or drug use, toxicity, epilepsy-related confusion, psychological or psychiatric-related amnesia (functional amnesia)": "other_cause",
    "Did not examine/report both retrograde and anterograde memory domains": "not_both_domains",
    "Without descriptive/qualitative/quantitative data on amnesia severity/memory tests/questions/scenarios/details": "not_enough_information",
    "Had global cognitive impairment disproportionate to memory loss": "disproportionate_impairment",
    "Without measurable lesion-related brain MR/CT scans": "no_scan",
    "Had focal or widespread brain atrophy": "neurodegenerative",
    "Atypical cases with selective (e.g., semantic) memory loss or material/topographic-specific memory loss": "atypical_case"
}
Here i

**Critical Note**
- You are going to define a dictionary with questions as keys (first) and outcomes as values (second).
- The value determines if you are answering a positive question or a negative question.
- If the question is positive (a yes is good), set the value to 1.
- If the question is negative (a yes is bad), set the value to 0.
- A good paper will be denoted by 1, with a bad paper denoted by 0.

**Examples**
```
question = {
    "Prioritizing implicit and explicit information, do you think this is a ______ case report or case series? (Yes/No)": 1,

    "Prioritizing implicit and explicit information, do you think this is a case of transient amnesia, reversible amnesia, confabulation, epilepsy, toxicity, neurodegenerative disease, or functional/psychiatric neurological disorder? . (Yes/No)": 0,

    "Prioritizing implicit and explicit information, do you think this examined both retrograde and anterograde amnesia? For example, if they report scores or a clinical examination examining retrograde and anterograde amnesia. (Yes/No)": 1,

    "Prioritizing implicit and explicit information, do you think this has some sort of qualitative or quantitative measurments on memory severity? For example, a case with neuropsychological measurements on memory tests. (Yes/No)": 1,

    "Prioritizing implicit and explicit information, do you think the memory loss might be due to global cognitive impairment? For example, a stroke resulting in executive, memory, language, and more changes is a global impairment. (Yes/No)": 0,

    "Prioritizing implicit and explicit information, do you think this case report has a figure with neuroimaging? (Yes/No)": 1,

    "Prioritizing implicit and explicit information, do you think this case report had brain atrophy, either focally or globally suggesting neurodegeneration? (Yes/No)": 0,

    "Prioritizing implicit and explicit information, do you think this is due to atypical memory loss, where only a subset of memory is impaired? For example, if just spatial memory is lost. (Yes/No)": 0,
    
    "Prioritizing implicit and explicit information, do you think this in English? (Yes/No)": 0,
}
```

In [4]:
question = {
    "Do you think this article may contain a case report or case series of a patient who lost their ability to use their mind's eye? Examples would be a stroke and a patient losing the ability to see colour. Another example would be a neurosurgical patient losing the ability to see shapes. Another example would be a patient who had brain damage and could no longer mentally visualize.": 1,
}

# 02 - Ask Questions

Define the segmented labels you want to consider. 

- Article type 'case' will has sections 'case_report' and 'other'
- Article type 'research' has sections "Abstract", "Introduction", "Methods", "Results", "Discussion", "Conclusion", "References"
- Article type 'inclusion' can use any keyword you want. It constrains the system to respond in a binary fashion: pass or fail for each question

In [5]:
# Define the keys you want to consider (exclude 'References')
keys_to_consider = [ "case_report", "other" ]  # Add or remove keys as per your requirement
article_type = 'inclusion'

Set test_mode=True during your first few runs, while you tune your questions to get the answers you need

In [6]:
test_mode=False

Submit Questions

In [7]:
from calvin_utils.gpt_sys_review.gpt_utils import OpenAIChatEvaluator
evaluator = OpenAIChatEvaluator(api_key_path=api_key_path, json_file_path=json_file_path, keys_to_consider=keys_to_consider, question_type=article_type, model_choice="gpt4",  question=question, test_mode=test_mode)
answers = evaluator.evaluate_all_files()
new_json_path = evaluator.save_to_json(answers)

Will evaluate only 1 articles for testing.


100%|██████████| 1/1 [00:03<00:00,  3.58s/it]

633 tokens counted by the OpenAI API. Estimated cost per article: 0.05697
Saved to: /Users/cu135/Dropbox (Partners HealthCare)/studies/Aphantasia_Kutsche/json/../case_extractions/inclusion_evaluations.json


# 03 - Summarize the Results

In [8]:
from calvin_utils.gpt_sys_review.json_utils import InclusionExclusionSummarizer
summarizer = InclusionExclusionSummarizer(new_json_path, questions=question)
result_df, raw_path, automated_path = summarizer.run()

Your CSV files of filtered manuscripts have been saved to this directory: 
 /Users/cu135/Dropbox (Partners HealthCare)/studies/Aphantasia_Kutsche/json/../case_extractions/inclusion_exclusion_results


/Users/cu135/.local/lib/python3.10/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


# 04 - Optional) Update Master List
- If you have been using a master_list, you can update it with the results from the generated CSVs.

In [9]:
master_list_path = '/Users/cu135/Dropbox (Partners HealthCare)/studies/Aphantasia_Kutsche/master_list.csv'

In [10]:
from calvin_utils.gpt_sys_review.txt_utils import PostProcessing
PostProcessing.add_raw_results_to_master_list(master_list_path=master_list_path, raw_results_path=raw_path)

Updating master list: 1it [00:00, 188.22it/s]


,PMID,Title,Authors,Citation,First author,Journal/book,Publication year,Create date,PMCID,Nihms id,DOI,Openai_screen,Abstract,OpenAI_Screen_Abstract,PDF_Downloaded,PDF_Path,Do you think this article may contain a case report or case series of a patient who lost their ability to use their mind's eye? Examples would be a stroke and a patient losing the ability to see colour. Another example would be a neurosurgical patient losing the ability to see shapes. Another example would be a patient who had brain damage and could no longer mentally visualize.
0,20079677,Dreaming and the brain: from phenomenology to ...,"Nir Y, Tononi G.",Trends Cogn Sci. 2010 Feb;14(2):88-100. doi: 1...,Nir Y,Trends Cogn Sci,2010,1/19/10,PMC2814941,NIHMS165848,10.1016/j.tics.2009.12.001,1,1. Trends Cogn Sci. 2010 Feb;14(2):88-100. doi...,0.0,NaN,NaN,NaN
1,27855424,"Arts, Brain and Cognition","Demarin V, Bedeković MR, Puretić MB, Pašić MB.",Psychiatr Danub. 2016 Dec;28(4):343-348.,Demarin V,Psychiatr Danub,2016,11/18/16,NaN,NaN,NaN,0,2. Psychiatr Danub. 2016 Dec;28(4):343-348.\n\...,NaN,NaN,NaN,NaN
2,25504833,Viewing the proteome: how to visualize proteom...,"Oveland E, Muth T, Rapp E, Martens L, Berven F...",Proteomics. 2015 Apr;15(8):1341-55. doi: 10.10...,Oveland E,Proteomics,2015,12/16/14,NaN,NaN,10.1002/pmic.201400412,0,3. Proteomics. 2015 Apr;15(8):1341-55. doi: 10...,NaN,NaN,NaN,NaN
3,23601759,Effectiveness of motor imagery or mental pract...,"García Carrasco D, Aboitiz Cantalapiedra J.",Neurologia. 2016 Jan-Feb;31(1):43-52. doi: 10....,García Carrasco D,Neurologia,2016,4/23/13,NaN,NaN,10.1016/j.nrl.2013.02.003,0,4. Neurologia. 2016 Jan-Feb;31(1):43-52. doi: ...,NaN,NaN,NaN,NaN
4,36950100,Study on insulin resistance and ischemic cereb...,"Zhou X, Kang C, Hu Y, Wang X.",Front Public Health. 2023 Mar 6;11:1021378. do...,Zhou X,Front Public Health,2023,3/23/23,PMC10025569,NaN,10.3389/fpubh.2023.1021378,0,5. Front Public Health. 2023 Mar 6;11:1021378....,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8447,32004786,[Borderline Ovarian Tumours: CNGOF Guidelines ...,"Canlorbe G, Lecointre L, Chauvet P, Azaïs H, F...",Gynecol Obstet Fertil Senol. 2020 Mar;48(3):28...,Canlorbe G,Gynecol Obstet Fertil Senol,2020,2/1/20,NaN,NaN,10.1016/j.gofs.2020.01.016,0,8448. Gynecol Obstet Fertil Senol. 2020 Mar;48...,NaN,NaN,NaN,NaN
8448,37430301,Spatial variations and influencing factors of ...,"Xiang L, Yamada M, Feng W, Li D, Nie H.",J Health Popul Nutr. 2023 Jul 11;42(1):66. doi...,Xiang L,J Health Popul Nutr,2023,7/10/23,PMC10334675,NaN,10.1186/s41043-023-00403-4,0,8449. J Health Popul Nutr. 2023 Jul 11;42(1):6...,NaN,NaN,NaN,NaN
8449,25854109,[Aiming for zero blindness],Nakazawa T.,Nippon Ganka Gakkai Zasshi. 2015 Mar;119(3):16...,Nakazawa T,Nippon Ganka Gakkai Zasshi,2015,4/10/15,NaN,NaN,NaN,0,8450. Nippon Ganka Gakkai Zasshi. 2015 Mar;119...,NaN,NaN,NaN,NaN
8450,31895223,Ultrafast Intracranial Vessel Imaging With Non...,"Sartoretti T, van Smoorenburg L, Sartoretti E,...",Invest Radiol. 2020 May;55(5):293-303. doi: 10...,Sartoretti T,Invest Radiol,2020,1/3/20,NaN,NaN,10.1097/RLI.0000000000000641,0,8451. Invest Radiol. 2020 May;55(5):293-303. d...,NaN,NaN,NaN,NaN


Your articles have been completely evaluated and filtered. 

Please check the CSVs in the directory noted above and use the path to the one you would like to use. It will be for your next notebook.
- Enjoy. If this has been helpful, please consider adding Calvin Howard as a collaborator. 
- e: choward12@bwh.harvard.edu